## Functions

In [1]:
def func1(x,y):
    return x if x < y else y

func2 = lambda x,y: x if x < y else y

class Func3:
    def __call__(self, x, y):
        return x if x < y else y
    
func3 = Func3()

func1(3,4) == func2(3,4) == func3(3,4)

True

In [2]:
i = 5
def f(arg = i):
    print(arg)

In [3]:
f()

5


In [4]:
f(4)

4


`def` statement is executable and defaults are evaluated once only when `def` is executed. This means that the expression (`def` statement) is evaluated once, when the function is defined, and that the same 'pre computed' value is used for each call.  

Below, each subsequent call to function keeps using the same list (as indicated by `id` function)

In [5]:
def f(l = []):
    print(id(l))
    l.append(2)
    print(l)


In [6]:
f()

416748996416
[2]


In [7]:
f()

416748996416
[2, 2]


Since default value is evaluated only once. This makes a difference when the default is a mutable object such as a list, dictionary, or instances of most classes. For example, the following function accumulates the arguments passed to it on subsequent calls:

In [8]:
def f(a, l = []):
    l.append(a)
    return l
print(f(1))
print(f(2))

[1]
[1, 2]



If you don't want that, write function like below - 

In [9]:
def f(a, l = None):
    if l is None:
        l = []
    l.append(a)
    return l
print(f(1))
print(f(2))

[1]
[2]


#### Mutable/Immutable argument to functions

In [10]:
def f(x,y):
    x = 23
    y.append(42)
    

a = 77
b = [99]
f(a,b)
print(a,b)


77 [99, 42]


Notice that `a` is still set to `77` while list `b` has changed. It happened because `a` is immutable while lists are mutable. 

Technically speaking, both `a` and list `b` haven't changed. Both are still pointing to the same objects they were pointing to earlier. It is just that content of list has changed because lists are mutable. Following code verify this:  

In [11]:
def f(x,y):
    x = 23
    y.append(42)
    
a = 77
b = [99]

print(id(a), id(b))

f(a,b)

print(a,b)

print(id(a), id(b))

416659624688 416746937216
77 [99, 42]
416659624688 416746937216


### Variable Scope in Function

In [20]:
def a1():
    a = 1
    
    def a2():
        a = 2
        
    a2()
    print(a)
    
a1()    

1


In [12]:
def f1(a):   
    def f2(b):
        b = a+b
        return b
    return f2    

x = f1(3)
print(x(4)) 

#here function f2 has access to 'a' from function f1.

7


This will also run-

In [16]:
def f1(a):
    def f2(b):
        c  =a+ b
        return c
    return f2
x = f1(5)
print(x(4)) 

9


But this will raise error-

In [18]:
def f1(a):
    def f2(b):
        a = a + b
        return a
    return f2
x = f1(5)
print(x(4)) 

UnboundLocalError: local variable 'a' referenced before assignment

So, we notice that the lines `b=a+b` in first example and `c=a+b` in second example ran just fine. However, `a=a+b` caused error. Notice the `UnboundLocalError` name. 

What is happening in these examples is that we are creating local variables `b`, `c` and `a`. In former two cases, we declared the variable and then assigned a value  to it. Using a variable which isn't assigned any value will raise `NameError`. However in third case, that is in `a=a+b`, Python regards `a` as newly created *local* variable. Now, since `a` is a local variable, the variable `a` defined in the outer function (that is, in enclosing scope) becomes *non-existent* as far as inner function is concerned. So, you defined a variable `a` in inner function scope and while assigning a value to it, you immediatedly tried to access its value (in `a+b`). Essentially, you are accessing something from `a` even before you assigned that thing to `a` and hence the error. In former cases of `b` and `c`, there wasn't any conflict. For example, while executing `c = a+b`, Python first looks for `a` in inner scope which isn't there so it then searches in enclosing scope and `a` is found there. In last case `a = a+b`, you are trying to declare a new variable `a` and assigning it a value which doesn't exist. 

To address this problem, we use `nonlocal a` as shown below. It tells Python that while executing `a = a + b`, we aren't creating a new local variable. Rather we are using the variable `a` defined in nonlocal (i.e. enclosing) scope.

[Source for above](https://stackoverflow.com/questions/2609518/unboundlocalerror-with-nested-function-scopes?utm_source=pocket_mylist)

Also note that - 

> Python evaluates expressions from left to right. Notice that while evaluating an assignment, the right-hand side is evaluated before the left-hand side.

In [12]:
def f1(a):
    def f2(b):
        nonlocal a   
        a = a + b
        return a
    return f2
x = f1(5)
print(x(4)) 


9


Notice the use of `nonlocal`. Due to this, function `f2` has access to `a` in function `f1`

Another interesting example, sourced from a [SO Post](https://stackoverflow.com/questions/49538724/why-is-ord-seen-as-an-unassigned-variable-here), shown below - 

In [3]:
def f():
    c = ord('a')
    return c

f()

97

In [9]:
if False:
    ord = None
def f():
    c = ord('a')
    return c

f()

97

In [16]:
def f():
    if False:
        ord = None 
    c = ord('a')
    
    return c


f()

UnboundLocalError: local variable 'ord' referenced before assignment

Since `if` clause is false and never executed, you'd think Python Interpreter doesn't take note of line `ord = None`. So in `c = ord('a')`, you'd assume, the built-in `ord()` will run. But that isn't the case as indicated by the error we get. More importantly, the `UnboundLocalError` tells us that Python sees `ord` as some local variable which hasn't been assigned any value. 

The explanation is that although `if` clause is never executed, Python does take note of `ord` local variable. It is just that it isn't assigned `None` value because condition is false. Hence, in next line, when `ord` is encountered again, Python recognize it from the earlier line where it wasn't assigned any value. Hence the error.

[Source for following example](https://docs.python.org/3/tutorial/classes.html#python-scopes-and-namespaces)

In [18]:
def scope_test():
    
    spam = "test spam"
    
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    #spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


### `locals` and `globals`

In [1]:
string1 = 'mayank'
def foo():
    string1 = 'mac'
    print(string1)
    print(locals())

foo()
globals()

mac
{'string1': 'mac'}


{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  "string1 = 'mayank'\ndef foo():\n    string1 = 'mac'\n    print(string1)\n    print(locals())\n\nfoo()\nglobals()"],
 '_oh': {},
 '_dh': ['C:\\Users\\pcxyz\\miniconda3\\Notebooks\\My notebooks\\Python\\Python Basics'],
 'In': ['',
  "string1 = 'mayank'\ndef foo():\n    string1 = 'mac'\n    print(string1)\n    print(locals())\n\nfoo()\nglobals()"],
 'Out': {},
 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x0000006C6121D280>>,
 'exit': <IPython.core.autocall.ZMQExitAutocall at 0x6c611fbe50>,
 'quit': <IPython.core.autocall.ZMQExitAutocall at 0x6c611fbe50>,
 '_': '',
 '__': '',
 '___': '',
 '_i': '',
 '_ii': '',
 '_iii': '',
 '_i1': "string1 = 'maya

### Examples of variable scoping

Now look at following different functions - 

In [15]:
b = 6
def f2(a):
    print(a)
    print(b)
f2(3)


3
6


In [16]:
b = 9
def f3(a):
    print(a)
    print(b)
    b = 6
f3(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

Compare last 2 examples.  

In [17]:
b =  4
def f4(a):
    global b
    print(a)
    print(b)
    b = 6

f4(3)    


3
4


### Function returning `None`: Be careful


In [18]:
>>> def divide(a,b):
	try:
		return a/b
	except ZeroDivisionError:
		return None
	
>>> x, y = 0, 5
>>> result = divide(x,y)     #result is 0
>>> if not result:           # 0 evaluates to False  
	print('invalid inputs')


invalid inputs


In conditional blocks, both `None` and `0` evaluates to `False`, so be careful here. In above example, `None` and 0 mean different things. 

#### Using Strings to Call Functions/Methods

In [3]:
>>> def bark():
	print('woof woof')

	
>>> def swim():
	print('splash!')

	
>>> dispatch = {'dog': bark, 'fish': swim}
>>> dispatch['dog']()
>>> dispatch['fish']()

woof woof
splash!


#### Using `locals()` or `eval()`

In [20]:
>>> def a():
	print('Hello')

	
>>> b =  'a'
>>> f = locals()[b]
>>> f()

Hello


In [21]:
>>> f = eval(b)
>>> f()


Hello


Create a script foo.py with following code - 
```Python
a = 47
def callfunc(func):
    return func
```
Now do this:

In [4]:
%%file foo.py

a = 47
def callfunc(func):
    return func


Writing foo.py


In [8]:
>>> from foo import callfunc
>>> a = 37
>>> def printf():
	return a
>>> callfunc(printf())

37

Notice the value of `a` returned. 

Note -  In the book ‘Python Essential Reference’(page 98), there were no parentheses after `printf` while calling `callfunc` function.

### Recursion

In [25]:
#example 1
def factorial(n):
    if n <= 1: return 1
    else: return n* factorial(n-1)

    
#example 2    
def flatten(lists):
    for s in lists:
        if isinstance(s,list):
            flatten(s)
        else:
            print(s)

print(factorial(4))
l = [1,2,[3,4,[5,6]]]
print(flatten(l));

24
1
2
3
4
5
6
None


However, be aware that there is a limit on the depth of recursive function calls. The function `sys.getrecursionlimit()` returns the current maximum recursion depth, and the function `sys.setrecursionlimit()` can be used to change the value.The default value is 1000. Although it is possible to increase the value, programs are still limited by the stack size limits enforced by the host operating system.When the recursion depth is exceeded, a `RuntimeError exception` is raised.

### Lambda Function

In [10]:
>>> def test(n):
	return lambda x: x+n

>>> a = test(4)
>>> a


<function __main__.test.<locals>.<lambda>(x)>

In [11]:
print(a(2))
print(a(3))

6
7


### `map` and `filter`

In [12]:
>>> list1 = [2,4,5]
>>> list2 = [3,6,7]
>>> map(lambda x,y: x+y, list1, list2)


In [13]:
>>> l = map(lambda x,y: x+y, list1, list2)
>>> list(l)

[5, 10, 12]

In [14]:
>>> a = filter(lambda x: x%2 ==0, list1)
>>> list(a)


[2, 4]

In [15]:
>>> a = filter(lambda x: x%2, list1)
>>> list(a)

[5]

In [6]:
>>> #sorting
>>> pairs = [(1,'one'),(2,'two'),(3,'three'),(4,'four')]
>>> pairs.sort(key = lambda pair : pair[1])
>>> pairs


[(4, 'four'), (1, 'one'), (3, 'three'), (2, 'two')]

### Lambda function gotcha

In [1]:
square = []
for x in range(5):
    square.append(lambda: x**2)
    
print(square)          #list of lambda functions
print(x)               # last value in range(5)
print(square[2]()) 
print(square[3]())


[<function <lambda> at 0x0000003A3D2CCDC0>, <function <lambda> at 0x0000003A3D2CC160>, <function <lambda> at 0x0000003A3D2CCD30>, <function <lambda> at 0x0000003A3D2CCCA0>, <function <lambda> at 0x0000003A3D2CCA60>]
4
16
16


In [2]:
>>> x = 9
>>> square[2]()


81

You’d expect that list `square` will be `[0,1,4,9,16]` but that is not the case. What we are actually getting is a list containing lambdas. In above case, `x` is not local to the lambdas, but is defined in the outer scope, and it is accessed when lambda is called – not when it is defined. Also, when we try to access an element from this list, we get the value of lambda function with `x =4` as final value of `x` in `range(5)` is `4` . For detail, refer Python Programming FAQ section. To avoid this, we modify above program –

In [3]:
 square = []
for x in range(5):
    square.append(lambda n = x: n**2)

print(square[2]())
print(square[3]())

4
9


Here, `n = x` creates a new variable `n` local to lambda and computed when the lambda is defined so that it has the same value that `x` had at that point in the loop. This means that the value of `n` will be 0 in the first lambda, 1 in the second and so on. Now, each lambda will return the correct result.

#### Gererator Expression

List comprehensions use square brackets, while generator expressions use parentheses.

In [36]:
[n ** 2 for n in range(5)]

[0, 1, 4, 9, 16]

In [37]:
(n ** 2 for n in range(5))

<generator object <genexpr> at 0x037B7A30>

In [38]:
list((n ** 2 for n in range(5)))

[0, 1, 4, 9, 16]

A list is a collection of values, while generator is a recipe for producing values. 

The difference is that a generator expression does not actually compute the values until they are needed. This not only leads to memory efficiency, but to computational efficiency as well! This also means that while the size of a list is limited by available memory, the size of a generator expression is unlimited!

A list can be itereated over multiple times, a generator expression is single use only.


In [39]:
g = (n ** 2 for n in range(5))
list(g)

[0, 1, 4, 9, 16]

In [40]:
list(g)

[]

#### Closure

In [57]:
def countdown(n):
    def nextf():
        nonlocal n
        r = n
        n -= 1
        return r
    return nextf
a = countdown(10)
while True:
	b = a()
	if not b: break
	print(b)


10
9
8
7
6
5
4
3
2
1


In this code, a closure is being used to store the internal counter value `n`. The inner function `nextf()` updates and returns the previous value of this counter variable each time it is called.
